In [4]:
import Processor.Order.Futures.Live
import API.Private.Futures
import API.Public.Futures
import Utils.TradingUtils as futures_utils

ins_orders = Processor.Order.Futures.Live.Orders()
ins_private_api = API.Private.Futures.API()
ins_public_api = API.Public.Futures.API()

symbols = ['ADAUSDT', 'DOGEUSDT']

balance = 1.2
leverage = 5
position_side = 'BUY'

for symbol in symbols:
    ins_orders.set_leverage(symbol, 2)
    ins_orders.set_margin_type(symbol, False)
    ticker_data = ins_public_api.fetch_mark_price(symbol)
    price = float(ticker_data['markPrice'])
    
    quantity = ins_orders.position_size(symbol, price, leverage, balance)
    
    imr = futures_utils.Calculator.imr(leverage)
    margin = futures_utils.Calculator.initial_margin(quantity['minSize'], price, imr)
    
    min_qty = quantity['minSize']
    
    ins_private_api.position_market_order(symbol, position_side, min_qty, "BOTH", False)
    # print("\n")
    # print(f"symbol: {symbol}")
    # print(f"quantity: {quantity["minSize"]}")
    # print(f"margin: {margin}")
    
    

# account_data = ins_private_api.fetch_account_balance()
# ins_orders.close_all_positions(account_data)

HTTPError: 400 Client Error: Bad Request for url: https://fapi.binance.com/fapi/v1/order?symbol=ADAUSDT&side=BUY&quantity=6.2&positionSide=BOTH&reduceOnly=False&type=MARKET&timestamp=1739521781520&signature=948a46e3dc23c275a9e59ab5d294c44ac478f47a8ce7264e3bdadef9db0d920d

In [ ]:
futures_utils.Calculator.initial_margin()